<a href="https://colab.research.google.com/github/ed-chin-git/DS-Unit-2-Sprint-3-Advanced-Regression/blob/master/DS_Unit_2_Sprint_Challenge_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 2 Sprint Challenge 3

## Logistic Regression and Beyond

In this sprint challenge you will fit a logistic regression modeling the probability of an adult having an income above 50K. The dataset is available at UCI:

https://archive.ics.uci.edu/ml/datasets/adult

Your goal is to:

1. Load, validate, and clean/prepare the data.
2. Fit a logistic regression model
3. Answer questions based on the results (as well as a few extra questions about the other modules)

Don't let the perfect be the enemy of the good! Manage your time, and make sure to get to all parts. If you get stuck wrestling with the data, simplify it (if necessary, drop features or rows) so you're able to move on. If you have time at the end, you can go back and try to fix/improve.

### Hints

It has a variety of features - some are continuous, but many are categorical. You may find [pandas.get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) (a method to one-hot encode) helpful!

The features have dramatically different ranges. You may find [sklearn.preprocessing.minmax_scale](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html#sklearn.preprocessing.minmax_scale) helpful!

### IMPORTS

In [0]:
### imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

## Part 1 - Load, validate, and prepare data

The data is available at: https://archive.ics.uci.edu/ml/datasets/adult

Load it, name the columns, and make sure that you've loaded the data successfully. Note that missing values for categorical variables can essentially be considered another category ("unknown"), and may not need to be dropped.

You should also prepare the data for logistic regression - one-hot encode categorical features as appropriate.


### Load and name the features
>50K, <=50K.

**age**: continuous.

**workclass**: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

**fnlwgt**: continuous.

**education**: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

**education-num**: continuous.

**marital-status**: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

**occupation**: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

**relationship**: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

**race**: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

**sex**: Female, Male.

**capital-gain**: continuous.

**capital-loss**: continuous.

**hours-per-week**: continuous.

**native-country**: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [104]:
data_url='https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
cols=['age','workclass','finalwgt','education','education-num' ,'marital-status','occupation','relationship' ,'race' ,'sex','capital_gain','capital_loss','hours_per_week','native_country','target' ]
adult_df=pd.read_csv(data_url,header=None, index_col=False,names=cols)
print(adult_df.shape)
adult_df.head()

(32561, 15)


,age,workclass,finalwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [0]:
adult_orig=adult_df.copy()

### One Hot Encode all categorical features  except native_country and target

In [106]:
cols_enc=['workclass','education','marital-status','occupation','relationship' ,'race' ,'sex']
adult_enc=pd.get_dummies(adult_df, columns=cols_enc, prefix=cols_enc)
adult_enc.head(10)


,age,finalwgt,education-num,capital_gain,capital_loss,hours_per_week,native_country,target,workclass_ ?,workclass_ Federal-gov,...,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Female,sex_ Male
0,39,77516,13,2174,0,40,United-States,<=50K,0,0,...,0,0,0,0,0,0,0,1,0,1
1,50,83311,13,0,0,13,United-States,<=50K,0,0,...,0,0,0,0,0,0,0,1,0,1
2,38,215646,9,0,0,40,United-States,<=50K,0,0,...,0,0,0,0,0,0,0,1,0,1
3,53,234721,7,0,0,40,United-States,<=50K,0,0,...,0,0,0,0,0,1,0,0,0,1
4,28,338409,13,0,0,40,Cuba,<=50K,0,0,...,0,0,1,0,0,1,0,0,1,0
5,37,284582,14,0,0,40,United-States,<=50K,0,0,...,0,0,1,0,0,0,0,1,1,0
6,49,160187,5,0,0,16,Jamaica,<=50K,0,0,...,0,0,0,0,0,1,0,0,1,0
7,52,209642,9,0,0,45,United-States,>50K,0,0,...,0,0,0,0,0,0,0,1,0,1
8,31,45781,14,14084,0,50,United-States,>50K,0,0,...,0,0,0,0,0,0,0,1,1,0
9,42,159449,13,5178,0,40,United-States,>50K,0,0,...,0,0,0,0,0,0,0,1,0,1


### Encode native_country

In [107]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#Auto encodes any dataframe column of type category or object.
def dummyEncode(df):
        columnsToEncode = list(df.select_dtypes(include=['category','object']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                df[feature] = le.fit_transform(df[feature])
            except:
                print('Error encoding '+feature)
        return df
 
dummyEncode(adult_enc)
adult_enc.head()

,age,finalwgt,education-num,capital_gain,capital_loss,hours_per_week,native_country,target,workclass_ ?,workclass_ Federal-gov,...,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Female,sex_ Male
0,39,77516,13,2174,0,40,39,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1,50,83311,13,0,0,13,39,0,0,0,...,0,0,0,0,0,0,0,1,0,1
2,38,215646,9,0,0,40,39,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,53,234721,7,0,0,40,39,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,28,338409,13,0,0,40,5,0,0,0,...,0,0,1,0,0,1,0,0,1,0


## Part 2 - Fit and present a Logistic Regression

Your data should now be in a state to fit a logistic regression. Use scikit-learn, define your `X` (independent variable) and `y`, and fit a model.

Then, present results - display coefficients in as interpretible a way as you can (hint - scaling the numeric features will help, as it will at least make coefficients more comparable to each other). If you find it helpful for interpretation, you can also generate predictions for cases (like our 5 year old rich kid on the Titanic) or make visualizations - but the goal is your exploration to be able to answer the question, not any particular plot (i.e. don't worry about polishing it).

It is *optional* to use `train_test_split` or validate your model more generally - that is not the core focus for this week. So, it is suggested you focus on fitting a model first, and if you have time at the end you can do further validation.

### Scale the data and run Logistic Regression

In [108]:
X = adult_enc.drop('target', axis='columns')
y = adult_enc['target']


# feature scaling for faster convergence
std_scaler = StandardScaler()
X = std_scaler.fit_transform(X)

adult_log1 = LogisticRegression(max_iter=500, multi_class='auto', solver='lbfgs').fit(X, y)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


### Score  :  Psuedo R2

In [109]:
adult_log1.score(X, y)

0.8520315715119314

### Here are the Coefficients and the intercept

In [110]:
adult_log1.coef_

array([[ 0.35388342,  0.06780737,  0.3772401 ,  2.34420446,  0.25946609,
         0.36702437,  0.05899665, -0.06633354,  0.10759884, -0.01246712,
        -0.08096716,  0.06116867,  0.05430117, -0.098355  , -0.03676908,
        -0.14803215, -0.08774348, -0.11213792, -0.03907604, -0.04590881,
        -0.04687143, -0.10310788, -0.07935634, -0.009791  ,  0.01993409,
         0.14620982,  0.11140767, -0.06878849,  0.13079146, -0.49536169,
         0.12397171,  0.02274824, -0.22491386,  0.05311106,  0.7527859 ,
        -0.07878043, -0.53323959, -0.13761931, -0.09377294, -0.07135188,
        -0.00657554, -0.01953396,  0.01593759,  0.25187036, -0.17402035,
        -0.14248551, -0.075418  , -0.25647515, -0.27459366,  0.16558768,
         0.07802566,  0.08022462,  0.10547967, -0.02787857, -0.06616481,
         0.17420972, -0.08959924, -0.2987819 ,  0.08071439,  0.25850998,
        -0.04525369,  0.00697803, -0.02174031, -0.04535354,  0.03882358,
        -0.19976024,  0.19976024]])

In [111]:
adult_log1.intercept_

array([-2.08358471])

### Data column names for reference to coefs

In [113]:
adult_enc.columns

Index(['age', 'finalwgt', 'education-num', 'capital_gain', 'capital_loss',
       'hours_per_week', 'native_country', 'target', 'workclass_ ?',
       'workclass_ Federal-gov', 'workclass_ Local-gov',
       'workclass_ Never-worked', 'workclass_ Private',
       'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc',
       'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 10th',
       'education_ 11th', 'education_ 12th', 'education_ 1st-4th',
       'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th',
       'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors',
       'education_ Doctorate', 'education_ HS-grad', 'education_ Masters',
       'education_ Preschool', 'education_ Prof-school',
       'education_ Some-college', 'marital-status_ Divorced',
       'marital-status_ Married-AF-spouse',
       'marital-status_ Married-civ-spouse',
       'marital-status_ Married-spouse-absent',
       'marital-status_ Never-married', 'marital-status

## Part 3 - Analysis, Interpretation, and Questions

### Based on your above model, answer the following questions

1. What are 3 features positively correlated with income above 50k?
           FEATURE..........................Coef
                Age ........................+0.3538
                Sex_male....................+0.1997
                workclass_ Federal-gov......+0.1075

2. What are 3 features negatively correlated with income above 50k?
                SEX_Female .................  -0.19976024
               marital-status_never_married. - 0.53323959
               education_ Preschool......... - 0.49536169
3. Overall, how well does the model explain the data and what insights do you derive from it?
               Pretty good at first glance.  the score (Psuedo R squared)=0.8520


*These answers count* - that is, make sure to spend some time on them, connecting to your analysis above. There is no single right answer, but as long as you support your reasoning with evidence you are on the right track.

Note - scikit-learn logistic regression does *not* automatically perform a hypothesis test on coefficients. That is OK - if you scale the data they are more comparable in weight.

### Match the following situation descriptions with the model most appropriate to addressing them

In addition to logistic regression, a number of other approaches were covered this week. Pair them with the situations they are most appropriate for, and briefly explain why.

Situations:
1. You are given data on academic performance of primary school students, and asked to fit a model to help predict "at-risk" students who are likely to receive the bottom tier of grades
                          use Quantile Regression 
                          your are modeling and estimating a specific threshold

2. You are studying tech companies and their patterns in releasing new products, and would like to be able to model and predict when a new product is likely to be launched.
                      use Survival Analysis  
                      you are trying to predict  the time of a certain event occurring.


3. You are working on modeling expected plant size and yield with a laboratory that is able to capture fantastically detailed physical data about plants, but only of a few dozen plants at a time.
                                   use Ridge Regression
                                   you have few observations and many features ( suffering from Overfitting)


Approaches:
1. Ridge Regression
2. Quantile Regression
3. Survival Analysis

**TODO - your answers!   See above please for answers**